In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
CHANNEL_NAME = os.getenv("CHANNEL_NAME")

# 슬랙 봇 토큰
SLACK_BOT_TOKEN = SLACK_BOT_TOKEN

In [12]:
from slack_sdk import WebClient

# Slack 클라이언트 초기화
client = WebClient(token=SLACK_BOT_TOKEN)  # 직접 토큰 입력 (환경 변수 대신)

def send_message(message: str, channel: str):
    try:
        client.chat_postMessage(channel=channel, text=message)
        print("메시지가 전송되었습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")

if __name__ == "__main__":
    send_message("안녕하세요, Slack 테스트 메시지입니다!", CHANNEL_NAME)


메시지가 전송되었습니다.


In [16]:
import os
from dotenv import load_dotenv
import python_bithumb as bit

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
BITHUMB_API_KEY = os.getenv("BITHUMB_API_KEY")
BITHUMB_SECRET_KEY = os.getenv("BITHUMB_SECRET_KEY")

In [ ]:
access_key = BITHUMB_API_KEY 
secret_key = BITHUMB_SECRET_KEY  

bithumb = bit.Bithumb(access_key, secret_key)

krw_balance = bithumb.get_balance("KRW")

send_message(
    f"""
        안녕하세요 오전 8시마다 보내드리는 알림입니다.
    ------------------------------------
    전체 자산 : {krw_balance}원

    전체 수익률 : 

    전날 대비 수익률 : 
    ------------------------------------        
    활기찬 연구생활 되시기를 바랍니다.
    """, CHANNEL_NAME
)

메시지가 전송되었습니다.


In [27]:
import os
import pandas as pd
import python_bithumb

def save_scalping_ohlcv_to_csv(
    ticker: str = "KRW-BTC",
    interval: str = "minute1",
    count: int = 100,
    filename: str = "scalping_data.csv"
):
    """
    스캘핑을 위해 짧은 봉(예: 1분봉)의 OHLCV 데이터를 조회하여 CSV로 저장하는 함수 예시.
    
    Parameters
    ----------
    ticker : str
        'KRW-BTC', 'KRW-ETH', 'KRW-XRP' 등 Bithumb 시장 티커
    interval : str
        'minute1', 'minute3', 'minute5', 'minute30' 등
        python_bithumb가 지원하는 분봉 단위
    count : int
        몇 개의 봉 데이터를 가져올지 설정 (스캘핑 시 100~200 정도 사용 가능)
    filename : str
        저장할 CSV 파일 경로(이름). 예: "btc_scalping_data.csv"
    
    Returns
    -------
    None
        지정한 파일로 CSV 저장 후 함수 종료
    """
    # 1) 데이터 가져오기
    df = python_bithumb.get_ohlcv(ticker, interval=interval, count=count)
    if df is None or df.empty:
        print(f"[WARNING] {ticker} {interval} 데이터가 비어 있습니다. 저장을 중단합니다.")
        return
    
    # 2) CSV 파일 저장할 폴더가 없으면 생성
    dir_name = os.path.dirname(filename)
    if dir_name and not os.path.exists(dir_name):
        os.makedirs(dir_name, exist_ok=True)
    
    # 3) CSV로 저장 (한글 깨짐 방지 위해 utf-8-sig 사용)
    #    index=True 또는 False 선택 가능. 여기서는 True로 저장해 날짜 인덱스 포함
    df.to_csv(filename, encoding='utf-8-sig', index=True)
    
    print(f"[INFO] '{ticker}'의 {interval} 데이터 {count}개를 '{filename}'에 저장했습니다.")


if __name__ == "__main__":
    # 1분봉 기준, 최근 100개 봉 데이터를 "btc_scalping_data.csv"에 저장
    save_scalping_ohlcv_to_csv(
        ticker="KRW-BTC",
        interval="minute1",
        count=100,
        filename="btc_scalping_data.csv"
    )

    # 필요에 따라 3분봉, 5분봉 등도 따로 저장할 수 있음
    save_scalping_ohlcv_to_csv(
        ticker="KRW-BTC",
        interval="minute3",
        count=100,
        filename="btc_scalping_data_3min.csv"
    )


[INFO] 'KRW-BTC'의 minute1 데이터 100개를 'btc_scalping_data.csv'에 저장했습니다.
[INFO] 'KRW-BTC'의 minute3 데이터 100개를 'btc_scalping_data_3min.csv'에 저장했습니다.


In [74]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

def crawl_with_selenium():
    """
    셀레니움으로 coinness.com/article 페이지 로딩 후
    특정 CSS 셀렉터로 지정된 하늘색 영역의 텍스트를 추출.
    """
    driver = webdriver.Chrome()
    driver.get("https://coinness.com/article")

    # JS 로딩 시간 대기
    time.sleep(5)  

    try:
        css = "#root > div > div.Wrap-sc-v065lx-0.hwmGSB > div > main > div.ContentContainer-sc-91rcal-0.jJHYjq > div.ArticleListContainer-sc-cj3rkv-0.fkxjqP"
        container = driver.find_element(By.CSS_SELECTOR, css)
        return container.text
    except Exception as e:
        print("요소를 찾지 못했습니다:", e)
        return ""
    finally:
        # 브라우저 종료
        driver.quit()

def parse_news_text(raw_text):
    """
    1) HH:MM (예: '00:13') 만나는 순간 새 기사 시작.
    2) 다음 줄 = 날짜
    3) 그 다음 줄 = 제목
    4) 그 뒤로 다음 HH:MM 전까지 = 본문
    """
    # 줄바꿈 기준으로 split 후, 공백만 있는 줄은 제거
    lines = [line.strip() for line in raw_text.splitlines() if line.strip()]
    
    time_pattern = re.compile(r'^\d{2}:\d{2}$')  # HH:MM
    articles = []
    
    current_time = None
    current_date = None
    current_title = None
    content_lines = []
    
    def save_article():
        # 현재까지 쌓인 기사 정보를 리스트에 추가
        if current_time and current_date and current_title:
            articles.append({
                "time": current_time,
                "date": current_date,
                "title": current_title,
                "content": "\n".join(content_lines).strip()
            })
    
    state = "idle"

    for line in lines:
        # 1) 새로운 시간 패턴 만남 -> 이전 기사 저장 후 새 기사 시작
        if time_pattern.match(line):
            save_article()  # 이전 기사 저장
            current_time = line
            current_date = None
            current_title = None
            content_lines = []
            state = "got_time"
            continue
        
        # 2) got_time 이후 첫 줄 -> 날짜
        if state == "got_time":
            current_date = line
            state = "got_date"
            continue
        
        # 3) got_date 이후 첫 줄 -> 제목
        if state == "got_date":
            current_title = line
            state = "got_title"
            continue
        
        # 4) 나머지 줄 -> 본문
        content_lines.append(line)
        state = "collecting_content"
    
    # 마지막 기사 저장
    save_article()
    return articles

def save_to_csv(articles, filename="my_news.csv"):
    """
    파싱된 기사를 [time, date, title, content] 컬럼 순서로 CSV 저장
    """
    if not articles:
        print("[INFO] 기사 리스트가 비어 있습니다.")
        return
    
    df = pd.DataFrame(articles, columns=["time", "date", "title", "content"])
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"[INFO] CSV 파일 '{filename}' 저장 완료.")

if __name__ == "__main__":
    # 1) Selenium 크롤링
    raw_text = crawl_with_selenium()
    
    # 2) 텍스트 파싱 -> 기사 리스트
    articles = parse_news_text(raw_text)
    
    # 3) CSV 저장
    save_to_csv(articles, "my_news.csv")


[INFO] CSV 파일 'my_news.csv' 저장 완료.


In [71]:
send_message(crawl_with_selenium(), CHANNEL_NAME)

메시지가 전송되었습니다.
